
This notebook demonstrates the full modeling pipeline for bankruptcy prediction: from preprocessing and feature engineering to model tuning and evaluation

⚠️ **Important:**  
The dataset used here is not the original dataset from which final results were derived.  
Due to differences in data quality and availability, this version may produce lower performance metrics.

✅ The original performance metrics, SHAP interpretations, and confusion matrices are available in:

📁 `\classification models results`

 A full explanation and discussion of results can be found in the results folder in the repository.

This notebook focuses on **methodology, not final performance**.


In [ ]:
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import (roc_auc_score, precision_score, recall_score, 
                             confusion_matrix, roc_curve, f1_score, precision_recall_curve,
                             classification_report)

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE  # (Not used here, but imported in case needed)
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif
import shap
from sklearn.metrics import classification_report

In [ ]:
raw_data = pd.read_excel("Metrics_Final_bankruptcy.xlsx")

In [17]:
raw_data.describe(include ="all")

,Lik_1,Lik_2,Lik_3,ROA,ROE,ROI,Rentabilita_trzieb,Rentabilita_nakladov,Ziskova_marza,Stupen_samofinancovania,Doba_splacania_zavazkov,Financna_paka,Obrat_aktiv,Firma,Rok,Bankrot
count,3820.000000,3820.000000,3820.000000,3916.000000,3917.000000,3916.000000,3676.000000,3836.000000,3676.000000,3916.000000,3.836000e+03,3917.000000,3916.000000,3918,3918.000000,3918.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,797,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENERGYR PLUS,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN
mean,35.329295,38.022343,39.188259,-22.958260,-4.397113,-22.469999,-464.094873,39.391871,-464.094873,6.311978,4.655308e+03,4.964372,1.987468,NaN,2019.666156,0.005360
std,1146.006218,1146.712966,1147.015282,700.141279,931.251218,699.333100,17886.300108,638.843015,17886.300108,559.255186,9.595439e+04,93.831809,6.361146,NaN,2.728051,0.073024
min,-396.853900,-737.769200,-728.743600,-22376.190500,-40395.744700,-22376.190500,-987800.000000,-8500.000000,-987800.000000,-18354.545500,0.000000e+00,-939.166700,-123.933800,NaN,2014.000000,0.000000
25%,0.183250,0.815175,1.109175,-1.699725,0.417400,-1.391200,-0.432875,-1.981300,-0.432875,16.759750,4.735355e+01,1.052000,0.659925,NaN,2018.000000,0.000000
50%,0.865450,1.868500,2.259150,4.181350,14.960300,4.608050,3.384200,3.954550,3.384200,51.713750,1.142148e+02,1.402600,1.362650,NaN,2020.000000,0.000000
75%,3.699925,5.869600,6.514375,16.160575,42.483300,16.453725,11.995850,16.461825,11.995850,84.520550,2.543437e+02,2.870000,2.263225,NaN,2022.000000,0.000000


**This code serves for creating bankruptcy prediction based on time, e.g. 1,2,3 years prior bankruptcy. However we do not use these newly created variables in our model training because there is not enough instances of bankruptcy in our dataset (we have tested it and did not come with acceptable results)**

In [ ]:
# Create a copy of raw_data to keep the original dataset intact
processed_data = raw_data.copy()

# Step 1: Identify the actual bankruptcy year for each company
firm_bankruptcy_years = processed_data.loc[processed_data["Bankrot"] == 1, ["Firma", "Rok"]].drop_duplicates()
firm_bankruptcy_years = firm_bankruptcy_years.set_index("Firma")["Rok"]  # Set Firm as index

# Step 2: Map the bankruptcy year to all rows of the same firm
processed_data["Bankruptcy_Year"] = processed_data["Firma"].map(firm_bankruptcy_years)

# Step 3: Compute 'Years_to_Bankruptcy'
processed_data["Years_to_Bankruptcy"] = processed_data["Bankruptcy_Year"] - processed_data["Rok"]

# Ensure `Bankruptcy_Year` is NOT used as a feature
# We only use it to generate labels, then remove it before training
processed_data["y_0y"] = (processed_data["Years_to_Bankruptcy"] == 0).astype(int)  # Bankruptcy year
processed_data["y_1y"] = (processed_data["Years_to_Bankruptcy"] == 1).astype(int)  # 1 year before
processed_data["y_2y"] = (processed_data["Years_to_Bankruptcy"] == 2).astype(int)  # 2 years before
processed_data["y_3y"] = (processed_data["Years_to_Bankruptcy"] == 3).astype(int)  # 3 years before

# Step 4: Drop unnecessary columns before training
# Keeping `Bankruptcy_Year` here for reference, but it will be removed before model training
processed_data.drop(columns=["Years_to_Bankruptcy", "Bankrot"], inplace=True)

# Step 5: Final Removal of `Bankruptcy_Year` to prevent leakage
# This should be done right before training
data1 = processed_data.drop(columns=["Bankruptcy_Year"])

# Step 6: Validate the Fix
print(data1[data1[["y_0y", "y_1y", "y_2y", "y_3y"]].sum(axis=1) > 0][["Firma", "Rok", "y_0y", "y_1y", "y_2y", "y_3y"]])


In [ ]:

# 1) DATA PREPARATION
drop_cols = ["y_0y", "y_1y", "y_2y", "y_3y", "Firma", "Rok"]
X = data1.drop(columns=drop_cols)
y = data1["y_0y"]

X_array = X.to_numpy()
X_winsorized_array = winsorize(X_array, limits=[0.01, 0.01]) #Optimal limit
X_winsorized = pd.DataFrame(X_winsorized_array, columns=X.columns)

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_winsorized, y, test_size=0.2, stratify=y, shuffle=True, random_state=42
)

# 2) SHIFT + LOG TRANSFORM - With this transformation we handle severe skewness of our features in dataset
def fit_log_transform_with_shift(df: pd.DataFrame):
    shift_map = {}
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        min_val = df[col].min()
        shift_map[col] = abs(min_val) + 1e-5 if min_val <= 0 else 0.0
    return shift_map

def transform_log_with_shift(df: pd.DataFrame, shift_map: dict):
    df_log = df.copy()
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_log[col] = np.log1p(df_log[col] + shift_map[col])
    return df_log

shift_map = fit_log_transform_with_shift(X_train_raw)
X_train_log = transform_log_with_shift(X_train_raw, shift_map)
X_test_log  = transform_log_with_shift(X_test_raw, shift_map)

# 3) KNN IMPUTATION (in log space) - Handling missing values 
knn_imputer = KNNImputer(n_neighbors=3)
X_train_imputed_log = knn_imputer.fit_transform(X_train_log)
X_test_imputed_log  = knn_imputer.transform(X_test_log)
X_train_imputed_log = pd.DataFrame(X_train_imputed_log, columns=X.columns)
X_test_imputed_log  = pd.DataFrame(X_test_imputed_log, columns=X.columns)

# 4) FINAL WINSORIZATION IN LOG SPACE -  fine-tunes the distribution, ensuring the log-transformed outliers don’t skew analysis
X_train_imputed_log_arr = X_train_imputed_log.to_numpy()
X_train_imputed_log_arr = winsorize(X_train_imputed_log_arr, limits=[0.01, 0.01])
X_train_final_log = pd.DataFrame(X_train_imputed_log_arr, columns=X.columns)

X_test_imputed_log_arr = X_test_imputed_log.to_numpy()
X_test_imputed_log_arr = winsorize(X_test_imputed_log_arr, limits=[0.01, 0.01])
X_test_final_log = pd.DataFrame(X_test_imputed_log_arr, columns=X.columns)

# 5) POLYNOMIAL FEATURES & FEATURE SELECTION - this will help us capture non-linear relationships
# Also interaction terms enables us to capture the joint effect of different pairs of independent variables on the dependent variable
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_final_log)
X_test_poly  = poly.transform(X_test_final_log)

selector = SelectKBest(f_classif, k=50) #We reduce dimensionality by keeping only 50 most relevant features
X_train_poly_sel = selector.fit_transform(X_train_poly, y_train)
X_test_poly_sel  = selector.transform(X_test_poly)

# Define selected_feature_names
poly_feature_names = poly.get_feature_names_out(X.columns)
selected_indices = selector.get_support()
selected_feature_names = poly_feature_names[selected_indices]
# Keep selected_feature_names as a numpy array, but ensure elements are strings
selected_feature_names = np.array([str(name) for name in selected_feature_names])
print("Selected Feature Names (first 5):", selected_feature_names[:5])  # Debug print to verify

# 6) MODEL TRAINING & COMPARISON - fine tuned using Bayesian optimization via Optuna library(not shown here)
models = {
    "XGBoost": XGBClassifier(eval_metric="logloss", seed=42, max_depth=6, learning_rate=0.01, n_estimators=700, 
                             subsample=1.0, colsample_bytree=0.8, scale_pos_weight=25, gamma=1),
    "RandomForest": RandomForestClassifier(random_state=42, n_estimators=700, max_depth=8, 
                                           min_samples_split=2, min_samples_leaf=2, bootstrap=True, class_weight='balanced'),
    "LightGBM": LGBMClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42)
}

results = {}

print("\n=== Model Comparison on Test Set ===\n")
for name, model in models.items():
    model.fit(X_train_poly_sel, y_train)
    y_prob = model.predict_proba(X_test_poly_sel)[:, 1]
    auc_val = roc_auc_score(y_test, y_prob)
    # Using a fixed threshold of 0.5 for now (will update in confusion matrix)
    y_pred = (y_prob >= 0.5).astype(int)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results[name] = {"AUC": auc_val, "Precision": prec, "Recall": rec, "F1": f1}
    print(f"{name}: {results[name]}")

# 7) Plot ROC Curves for All Models
plt.figure(figsize=(8, 6))
for name, model in models.items():
    y_prob = model.predict_proba(X_test_poly_sel)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc_val = roc_auc_score(y_test, y_prob)
    plt.plot(fpr, tpr, label=f"{name} (AUC={auc_val:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Bankruptcy Prediction")
plt.legend(loc="lower right")
plt.show()

# 8) Threshold Tradeoff Function & Tuning for Each Model - with this approach we explore how different treshold influence precision and recall tradeoff
def tune_threshold(y_true, y_prob):
    # Explore thresholds from 0.0 to 1.0 in 0.01 increments.
    thresholds = np.linspace(0, 1, 101)
    best_f1 = 0
    best_thresh = 0
    f1_scores = []
    precision_scores = []
    recall_scores = []
    for thresh in thresholds:
        y_pred = (y_prob >= thresh).astype(int)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
    return best_thresh, best_f1, thresholds, f1_scores, precision_scores, recall_scores

for name, model in models.items():
    y_prob = model.predict_proba(X_test_poly_sel)[:, 1]
    best_thresh, best_f1, thresh_arr, f1_arr, prec_arr, rec_arr = tune_threshold(y_test, y_prob)
    
    plt.figure(figsize=(8, 6))
    plt.plot(thresh_arr, f1_arr, "r-", label="F1 Score")
    plt.plot(thresh_arr, prec_arr, "b--", label="Precision")
    plt.plot(thresh_arr, rec_arr, "g-.", label="Recall")
    plt.axvline(x=best_thresh, color="k", linestyle="--", label=f"Best Threshold = {best_thresh:.2f}")
    plt.xlabel("Threshold")
    plt.ylabel("Metric Value")
    plt.title(f"Threshold Tradeoff Curve ({name})")
    plt.legend(loc="upper left")
    plt.ylim([0, 1])
    plt.show()
    
    print(f"{name} optimal threshold (max F1): {best_thresh:.2f} with F1 = {best_f1:.3f}")

# 9) PRINT FINAL COMPARISON TABLE (Using Fixed Threshold=0.5)
comparison_df = pd.DataFrame(results).T
print("\n=== Final Model Comparison (Fixed Threshold=0.5) ===")
print(comparison_df)

# 10) Classification Reports & Modified SHAP Summary for Top 10 Features

for name, model in models.items():
    y_prob = model.predict_proba(X_test_poly_sel)[:, 1]
    # Use fixed 0.5 threshold for classification report
    y_pred = (y_prob >= 0.5).astype(int)
    print(f"\n=== Classification Report for {name} ===")
    print(classification_report(y_test, y_pred, target_names=["Non-Bankrupt", "Bankrupt"]))
    
    print(f"\n--- SHAP Summary for {name} (Top 10 Features) ---")
    # Use TreeExplainer for tree-based models; use LinearExplainer for LogisticRegression
    if name in ["XGBoost", "RandomForest", "LightGBM"]:
        explainer = shap.TreeExplainer(model, feature_perturbation="tree_path_dependent")
        shap_values = explainer.shap_values(X_train_poly_sel)
    elif name == "LogisticRegression":
        try:
            explainer = shap.LinearExplainer(model, X_train_poly_sel, feature_perturbation="interventional")
            shap_values = explainer.shap_values(X_train_poly_sel)
        except Exception as e:
            print("Error computing SHAP for LogisticRegression:", e)
            continue
    
    # For tree-based models, shap_values can be a list [class0, class1], so pick the positive class
    if isinstance(shap_values, list):
        shap_values = shap_values[1]  # Positive class
    elif len(shap_values.shape) == 3:
        # Some models (e.g., RF) might return (n_samples, n_features, n_classes)
        shap_values = shap_values[:, :, 1]  # Select positive class
    
    print(f"SHAP values shape for {name}: {shap_values.shape}")
    
    # Produce a SHAP summary plot (violin) for top 10 features
    # This automatically shows the 10 most important features.
    shap_df = pd.DataFrame(X_train_poly_sel, columns=selected_feature_names)
    shap.summary_plot(shap_values, shap_df, plot_type="violin", max_display=10)


# 11) Confusion Matrix for Each Model with Custom Thresholds
print("\n=== Confusion Matrices for Each Model ===")

# Define custom thresholds for each model
thresholds = {
    "XGBoost": 0.80,
    "RandomForest": 0.31,
    "LightGBM": 0.42,
    "LogisticRegression": 0.08
}

for name, model in models.items():
    # Get predicted probabilities
    y_prob = model.predict_proba(X_test_poly_sel)[:, 1]
    # Use custom threshold for this model
    thresh = thresholds[name]
    y_pred = (y_prob >= thresh).astype(int)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=["Non-bankrupt", "Bankrupt"], 
                yticklabels=["Non-bankrupt", "Bankrupt"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix ({name})")
    # Add metrics as text above the matrix
    auc_val = roc_auc_score(y_test, y_prob)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    plt.text(0.5, 1.05, f"{name}\nAUC: {auc_val:.3f}\nThreshold: {thresh:.2f}\nPrecision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}", 
             transform=plt.gca().transAxes, ha='center', va='bottom', fontsize=10)
    plt.tight_layout()
    plt.show()
    
    print(f"{name} Confusion Matrix:\n{cm}")

# Practical example of predicting probability of bankruptcy of a given company - this approach step is optional, we do this only when we need prediction for exactly one company from our list

In [ ]:

XGB = XGBClassifier(eval_metric="logloss", seed=42, max_depth=6, learning_rate=0.01, n_estimators=700, 
                             subsample=1.0, colsample_bytree=0.8, scale_pos_weight=25, gamma=1)
XGB.fit(X_train_final_log, y_train)

In [ ]:
RF = RandomForestClassifier(random_state=42, n_estimators=700, max_depth=8,min_samples_split=2, min_samples_leaf=2, bootstrap=True, class_weight='balanced')
RF.fit(X_train_final_log, y_train)

In [ ]:
LGB = LGBMClassifier(random_state=42)
LGB.fit(X_train_final_log, y_train)

In [11]:
LR = LogisticRegression(max_iter=1000, random_state=42)
LR.fit(X_train_final_log, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
final_predict = pd.read_excel("given company financial metrics.xlsx")
X_train_final_log = final_predict
probabilities = XGB.predict_proba(X_train_final_log)
print(probabilities)

In [ ]:
probabilities = RF.predict_proba(X_train_final_log)
print(probabilities)

In [ ]:
probabilities = LGB.predict_proba(X_train_final_log)
print(probabilities)

In [ ]:
probabilities = LR.predict_proba(X_train_final_log)
print(probabilities)

In [ ]:
import pickle
#Saving models
filename = 'XGB_class.sav'
pickle.dump(XGB, open(filename, 'wb'))
filename = 'RF_class.sav'
pickle.dump(RF, open(filename, 'wb'))
filename = 'HGB_class.sav'
pickle.dump(LGB, open(filename, 'wb'))